In [1]:
import functions 
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np

import os 
import glob

import seaborn as sns 

# for all the month, calculate Q100 and dfs in mm 

In [2]:
# calculate volume in mm - should be the same function as for hydrology 

In [3]:
def sediment_volume_in_mm(sed, column, area=4830.0):
    """
    """
    # Ensure `D` column is in datetime format
    sed['D'] = pd.to_datetime(sed['D'])

    sed[column] = hydro[[sed]]
    
    # Set 'D' as the index
    sed = sed.set_index('D')
    
    # Resample to monthly data and calculate the sum for each month for the specified column
    sed_month = sed[column].resample('M').sum().reset_index()
    
    # Extract year and month for grouping or further analysis
    sed_month['year'] = sed_month['D'].dt.year
    sed_month['month'] = sed_month['D'].dt.month
    
    return sed_month




def water_balance_components_in_mm(hydro, column, area=4830.0):
    """
    Calculate the sum of water or sediment volume per month for a specific column, 
    and return the mean volumes across years.
    
    Parameters:
    hydro (pd.DataFrame): Input DataFrame with columns `D` (date) and the specified numeric column.
    column (str): The name of the column to calculate the volume for.
    area (float): Area in square meters used to scale the volumes. Default is 4830.0.
    
    Returns:
    pd.DataFrame: DataFrame with the calculated monthly volumes.
    """
    # Ensure `D` column is in datetime format
    hydro['D'] = pd.to_datetime(hydro['D'])

    if column == 'snowmelt':
        hydro['snowmelt'] = hydro['snowacc'].where(hydro['snowacc'] < 0).fillna(0)
    if column == 'snowacc2':
        hydro['snowacc2'] = hydro['snowacc'].where(hydro['snowacc'] > 0).fillna(0)
    if column == 'rainfall':
        hydro['rainfall'] = hydro['Pr'] - hydro['snowacc']
        hydro['rainfall'] = hydro['rainfall'].fillna(0)
        
    
    # Scale the specified column by the area
    hydro[column] = hydro[[column]]
    
    # Set 'D' as the index
    hydro = hydro.set_index('D')
    
    # Resample to monthly data and calculate the sum for each month for the specified column
    hydro_month = hydro[column].resample('M').sum().reset_index()
    
    # Extract year and month for grouping or further analysis
    hydro_month['year'] = hydro_month['D'].dt.year
    hydro_month['month'] = hydro_month['D'].dt.month
    
    return hydro_month

# 

In [61]:
%%time


percentile = '60percent'
method = 'once'

location = 'mustang'


if location == 'langtang':
    folder_loc = '12a'
if location == 'mustang': 
    folder_loc = '13a'



output_path = f'/Users/varyabazilova/Desktop/paper2/202406_modelruns/30years/2025May_output/SL_{method}/output_{percentile}'

# Define the list of folder paths

# '/Volumes/T7 Shield/202409_paper2_modelruns/30years/SL_daily/1landcover_20percent'

folder_paths = [f'/Volumes/T7 Shield/202409_paper2_modelruns/May2025_30years/SL_{method}/1landcover_{percentile}/{location}_climate_cut',
                f'/Volumes/T7 Shield/202409_paper2_modelruns/May2025_30years/SL_{method}/2landcover_{percentile}/{location}_climate_cut',
                f'/Volumes/T7 Shield/202409_paper2_modelruns/May2025_30years/SL_{method}/3landcover_{percentile}/{location}_climate_cut',
                f'/Volumes/T7 Shield/202409_paper2_modelruns/May2025_30years/SL_{method}/4landcover_{percentile}/{location}_climate_cut',
                f'/Volumes/T7 Shield/202409_paper2_modelruns/May2025_30years/SL_{method}/5landcover_{percentile}/{location}_climate_cut']


# Load elevation data and define the output path
elevation = pd.read_csv(f'/Users/varyabazilova/Desktop/paper2/downscaling_simple/coordinates_and_elevation_with_labels_{location}.csv')[['cellnr2', 'band_data']]
elevation = elevation.transpose()
elevation_list = elevation.loc['cellnr2'].tolist() 

CPU times: user 10.8 ms, sys: 3.3 ms, total: 14.1 ms
Wall time: 15.3 ms


In [62]:
columns = ['dfs'] 


In [63]:

%%time
print(columns)  # Ensure `columns` is defined as a list of column names

# Iterate over columns
for column in columns:
    print(f"Processing column: {column}")

    # Iterate over folder paths
    for folder_path in folder_paths:
        print(f"Processing folder: {folder_path}")

        output_folder = output_path

        # Extract "1landcover" from the folder path
        landcover_part = os.path.basename(os.path.dirname(folder_path))
        landcover_base = landcover_part.split('_')[0]  # This gives "1landcover"

        # Load timestep DataFrame (assuming Hydro.out exists in the `/12a/` subfolder for all folders)
        hydro_path = os.path.join(folder_path, folder_loc, 'Sediment.out')
        print(f"Checking Sediment.out path: {hydro_path}")

        if not os.path.isfile(hydro_path):
            print(f"Sediment.out not found in {hydro_path}")
            continue  # Skip this folder_path if the main Hydro.out is missing

        # Load the Hydro.out file for timestep calculation
        try:
            dfts = pd.read_csv(hydro_path)
            df = water_balance_components_in_mm(dfts, column, area=4830.0)
            # df = sediment_volume_in_mm(dfts, column, area=4830.0)
            df = df[['month', 'year']]
        except Exception as e:
            print(f"Error loading timestep file: {e}")
            continue

        # Initialize the result DataFrame for this folder_path
        result_df = pd.DataFrame()

        # Iterate over subfolders in the folder path
        for folder_name in os.listdir(folder_path):
            subfolder_path = os.path.join(folder_path, folder_name)
            
            # Skip files and hidden folders
            if not os.path.isdir(subfolder_path) or folder_name.startswith('.'):
                continue

            file_path = os.path.join(subfolder_path, 'Sediment.out')
            # print(f"Checking subfolder Hydro.out: {file_path}")

            # Ensure the Hydro.out file exists in the subfolder
            if os.path.isfile(file_path):
                try:
                    hydro = pd.read_csv(file_path)
                    output_df = water_balance_components_in_mm(hydro, column)

                    # Extract and rename the target column
                    name_column = output_df[column]
                    column_name = f'{column}_{folder_name}'
                    result_df[column_name] = name_column
                except Exception as e:
                    print(f"Error processing file {file_path}: {e}")
                    continue

        # Ensure result_df has data before proceeding
        if result_df.empty:
            print(f"No valid data found in subfolders of {folder_path}")
            continue

        # Rename columns for clarity and filter by elevation list
        result_df.columns = [col[-3:] for col in result_df.columns]  # Adjust column names if necessary
        result_df = result_df[elevation_list]

        # Check for consistency with the elevation list
        if elevation_list == result_df.columns.tolist():
            print("Default land cover. Monthly data: same")

            # Rename columns, merge with timestep, and add folder info
            result_df.columns = elevation.loc['band_data']
            result_df = pd.concat([df, result_df], axis=1)
            result_df['folder'] = os.path.basename(folder_path)

            # print(result_df)

            # Define the output file name and save the DataFrame to a CSV file
            output_filename = f'{location}_monthly_sum_elevation_{column}_{percentile}_{landcover_base}_mm.csv'
            print('output filename:', output_filename)
            result_df.to_csv(os.path.join(output_folder, output_filename), index=False)

            print(f"Saved {output_filename}")
        else:
            print(f"The data does not match the elevation list for {folder_path}!")



['dfs']
Processing column: dfs
Processing folder: /Volumes/T7 Shield/202409_paper2_modelruns/May2025_30years/SL_once/1landcover_60percent/mustang_climate_cut
Checking Sediment.out path: /Volumes/T7 Shield/202409_paper2_modelruns/May2025_30years/SL_once/1landcover_60percent/mustang_climate_cut/13a/Sediment.out


<timed exec>:58: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


Default land cover. Monthly data: same
output filename: mustang_monthly_sum_elevation_dfs_60percent_1landcover_mm.csv
Saved mustang_monthly_sum_elevation_dfs_60percent_1landcover_mm.csv
Processing folder: /Volumes/T7 Shield/202409_paper2_modelruns/May2025_30years/SL_once/2landcover_60percent/mustang_climate_cut
Checking Sediment.out path: /Volumes/T7 Shield/202409_paper2_modelruns/May2025_30years/SL_once/2landcover_60percent/mustang_climate_cut/13a/Sediment.out
Default land cover. Monthly data: same
output filename: mustang_monthly_sum_elevation_dfs_60percent_2landcover_mm.csv
Saved mustang_monthly_sum_elevation_dfs_60percent_2landcover_mm.csv
Processing folder: /Volumes/T7 Shield/202409_paper2_modelruns/May2025_30years/SL_once/3landcover_60percent/mustang_climate_cut
Checking Sediment.out path: /Volumes/T7 Shield/202409_paper2_modelruns/May2025_30years/SL_once/3landcover_60percent/mustang_climate_cut/13a/Sediment.out
Default land cover. Monthly data: same
output filename: mustang_mont